In [5]:
import pandas as pd
import numpy as np

In [23]:
OScolnames = {'object_number': 'category',
              'obs_type': 'category',
              'measure_type': 'category',
              '?': 'category',
              'year': np.uint16,
              'month': np.uint8,
              'day': np.float32,
              'date_accuracy': np.float32,
              'RA_HH': np.uint8,
              'RA_mm': np.uint8,
              'RA_ss': np.float32,
              'RA_accuracy': np.float32,
              'RA_RMS': np.float32,
              #'RA_F': 'category',
              'RA_bias': np.float32,
              'RA_delta': np.float32,
              'DEC_HH': np.uint8,
              'DEC_mm': np.uint8,
              'DEC_ss': np.float32,
              'DEC_accuracy': np.float32,
              'DEC_RMS': np.float32,
              #'DEC_F': 'category',
              'DEC_bias': np.float32,
              'DEC_delta': np.float32,
              'MAG': np.float32,
              'MAG_B': 'category',
              'MAG_RMS': np.float32,
              'MAG_resid': np.float32,
              'catalog': 'category',
              'obs_code': 'category',
              'xhi_square': np.float32,
              'acceptance': 'bool',
              'mag_acceptance': 'bool'}
c=[(0, 11), (11, 13), (13, 15), (15, 17), (17, 21), (22, 25), (25, 40), (40, 50), (50, 53), (53, 56), (56, 64), (64, 77), (77, 83), (87, 96), (96, 103), (103, 107), (107, 110), (110, 117), (117, 130), (128, 136), (141, 150), (150, 156), (156, 161), (161, 164), (164, 170), (170, 178), (178, 180), (180, 186), (189, 194), (194, 196), (196, 197)]

OSdata = pd.read_fwf(r'Data\NEODYS_cleaned.txt', header=None, names=list(OScolnames.keys()), dtype={'obs_code': 'category', 'measure_type': 'category'}, colspecs=c, low_memory=True, index_col=False, usecols=['year', 'obs_code', 'acceptance', 'measure_type', 'RA_delta', 'DEC_delta'], error_bad_lines=False)

OSdata['delta_vect'] = np.sqrt(np.power(OSdata.RA_delta,2) + np.power(OSdata.DEC_delta,2))

In [69]:
types_dict = {'C':'CCD',
              'S':'Space observation',
              'A':'Observations from B1950.0 converted to J2000.',
              'c':'Corrected without republication CCD observation',
              'P': 'Photographic',
              'T':'Meridian or transit circle ',
              'X':'Discovery observation ',
              'x':'Discovery observation ',
              'M':'Micrometer ',
              'H':'Hipparcos geocentric observation ',
              'R':'Radar Observation',
              'E':'Occultations derived observation ',
              'V':'Roving observer observation',
              'n':'Mini-normal place derived from averaging observations from video frames',
              'e':'Encoder ' }

In [70]:
types = list(OSdata['measure_type'].value_counts().index)
table = pd.DataFrame(index=types, columns=['description', 'count', 'percentage', 'acceptance', 'timespan'])
for t in types:
    description = types_dict[t]
    count = len(OSdata[(OSdata['measure_type'] == t)].index)
    percentage = count / len(OSdata.index) * 100
    acceptance = (len(OSdata[(OSdata['measure_type'] == t) & (OSdata['acceptance']==True)]) / count * 100) if count > 0 else 'x'
    timespan = str(OSdata[OSdata['measure_type'] == t].year.min()) + '-' + str(OSdata[OSdata['measure_type'] == t].year.max())
    table.loc[t] = pd.Series({'description':description, 'count':count, 'percentage':percentage, 'acceptance':acceptance, 'timespan':timespan})

In [71]:
table['count'] = table['count'].astype(int)
table

,description,count,percentage,acceptance,timespan
C,CCD,2444198,97.1931,99.3748,1990-2020
S,Space observation,44493,1.76926,99.982,1994-2020
A,Observations from B1950.0 converted to J2000.,14125,0.561678,98.9593,1893-1998
c,Corrected without republication CCD observation,5308,0.211072,97.5885,1991-2013
P,Photographic,3955,0.15727,100,1898-2017
n,Mini-normal place derived from averaging obser...,1736,0.0690317,99.8848,2009-2020
M,Micrometer,642,0.025529,100,1898-1935
V,Roving observer observation,134,0.00532849,86.5672,2001-2020
x,Discovery observation,101,0.00401625,100,2010-2010
T,Meridian or transit circle,59,0.00234612,100,1988-1995


In [68]:
space_telescopes = {'Spitzer Space Telescope':'245', 
                    'Hubble Space Telescope':'250', 
                    'Gaia':'258', 
                    'WISE':'C51', 
                    'NEOSSat':'C53', 
                    'Kepler':'C55', 
                    'TESS':'C57'}
ST_df = pd.DataFrame(index=space_telescopes.keys(), columns=['count', 'acceptance'])
for name, obs in space_telescopes.items():
    count = len(OSdata[(OSdata['obs_code']==obs) & (OSdata['measure_type']=='S')])
    acceptance = (len(OSdata[(OSdata['obs_code']==obs) & (OSdata['acceptance']==True) & (OSdata['measure_type']=='S')]) / count * 100) if count > 0 else 'x'
    ST_df.loc[name] = pd.Series({'count':count, 'acceptance':acceptance})
ST_df


,count,acceptance
Spitzer Space Telescope,0,x
Hubble Space Telescope,2,100
Gaia,0,x
WISE,39479,99.9823
NEOSSat,3,100
Kepler,16,100
TESS,4993,99.98


In [73]:
table.to_csv('obs_type_stats.csv')
table.to_clipboard()
ST_df.to_csv('space_telescope_stats.csv')